In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
df = pd.read_excel("../data/raw/Reporte SSMOdigital Convenios.xlsx")

## Identificar la procedencia de cada uno de los movimientos de un convenio

En este apartado se quiere identificar la unidad en donde se ha realizado un movimiento a cada
convenio.